In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Conv1D,MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import to_categorical
import h5py

/home/river/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from timeit import default_timer as timer

import numpy as np
import math
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from keras_helper import NNWeightHelper
from snes import SNES
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [3]:
img_width, img_height = 28, 28
train_data_dir = "/home/river/assi2_ce888/Offical31_datasets/amazon/images"
validation_data_dir = "/home/river/assi2_ce888/Offical31_datasets/dslr/images"
#validation_data_dir = "/home/river/assi2_ce888/Offical31_datasets/webcam/images"


In [9]:
nb_train_samples = 527
nb_validation_samples = 423
epochs = 30
batch_size = 160
num_classes = 31
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
#input_shape = 31


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    save_to_dir= "/home/river/assi2_ce888/Offical31_datasets/save_pic1",
    target_size=(img_width, img_height),
    #color_mode = "grayscale",
    batch_size=batch_size,
    class_mode='categorical')

#train_generator = np.vstack([train1_generator, train2_generator])
#train_generator =zip(train1_generator,train2_generator)
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    #color_mode ="grayscale",
    batch_size=batch_size,
    class_mode='categorical')

#model.save_weights('train_data.h5py')

Found 2817 images belonging to 31 classes.
Found 498 images belonging to 31 classes.


In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(31))
model.add(Activation('softmax'))

In [ ]:
#model = Sequential()
model.add(Conv1D(32, 3,input_shape= (2817,31)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(32, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(64, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(31))
model.add(Activation('softmax'))

In [12]:
model.compile(loss = "categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_imgs, train_labels,  
          epochs=epochs,  
          batch_size=batch_size,  
          validation_data=(test_imgs, test_labels))  

In [13]:
nb_train_samples = len(train_generator.filenames)  
   
predict_size_train = int(math.ceil(nb_train_samples / batch_size))  

train_feature = model.predict_generator(train_generator, predict_size_train)  
   
np.save('train_feature.npy', train_feature)  
#model.save_weights("train_data.h5py", train_feature)

In [29]:
print(predict_size_train)

18


In [14]:
nb_test_samples = len(validation_generator.filenames)  
   
predict_size_test = int(math.ceil(nb_test_samples / batch_size))  

test_feature = model.predict_generator(validation_generator, predict_size_test)  
   
np.save('test_feature.npy', test_feature)  

In [15]:
train_imgs = np.load('train_feature.npy')  
     
train_labels = train_generator.classes  
   
train_labels = to_categorical(train_labels, num_classes=num_classes) 

In [16]:
test_imgs = np.load('test_feature.npy')  
   
test_labels = validation_generator.classes  
   
test_labels = to_categorical(test_labels, num_classes=num_classes) 

In [21]:
print(train_imgs.shape)

(2817, 31)


In [ ]:
#train_imgs, train_label= train_generator.next()
#test_imgs, test_label = validation_generator.next()

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
#score ,acc= model.evaluate_generator(validation_generator, nb_validation_samples // batch_size)

In [18]:
SAMPLE_SIZE = 1024
# how many different sets of weights ask() should return for evaluation
POPULATION_SIZE = 10
# how many times we will loop over ask()/tell()
GENERATIONS = 30

def train_classifier(model, X,y):
    
    #X_features = model.predict_generator(X, predict_size_train)  
    X_features = model.predict(X)
    clf = ExtraTreesClassifier(n_estimators=100, n_jobs=4)
    #clf = DecisionTreeClassifier()
    #clf = RandomForestClassifier(n_estimators=100, n_jobs=4)

    clf.fit(X_features,y)
    y_pred = clf.predict(X_features)
    return clf, y_pred


def predict_classifier(model, clf, X):
    #X_features = model.predict_generator(X,predict_size_train)
    X_features = model.predict(X)
    return clf.predict(X_features)

In [19]:
nnw = NNWeightHelper(model)
weights = nnw.get_weights()

In [20]:
def main():
    print("Total number of weights to evolve is:", weights.shape)

    all_examples_indices = list(range(train_imgs.shape[0]))
    print(len(all_examples_indices))

    clf, _ = train_classifier(model, train_imgs, train_labels)
    print("train_imgs shape is :", train_imgs.shape)
    print("train_labels shape is :",train_labels.shape)
    y_pred = predict_classifier(model, clf, test_imgs)
    print(test_labels.shape, y_pred.shape)
    test_accuracy = accuracy_score(test_labels, y_pred)

    print('Non-trained NN Test accuracy:', test_accuracy)
    # print('Test MSE:', test_mse)

    snes = SNES(weights, 1, POPULATION_SIZE)
    for i in range(0, GENERATIONS):
        start = timer()
        asked = snes.ask()

        # to be provided back to snes
        told = []
        # use a small number of training samples for speed purposes
        subsample_indices = np.random.choice(all_examples_indices, size=SAMPLE_SIZE)
        # evaluate on another subset
        subsample_indices_valid = np.random.choice(all_examples_indices, size=SAMPLE_SIZE + 1)

        # iterate over the population
        for asked_j in asked:
            # set nn weights
            nnw.set_weights(asked_j)
            # train the classifer and get back the predictions on the training data
            clf, _ = train_classifier(model, train_imgs[subsample_indices], train_labels[subsample_indices])

            # calculate the predictions on a different set
            y_pred = predict_classifier(model, clf, train_imgs[subsample_indices_valid])
            score = accuracy_score(train_labels[subsample_indices_valid], y_pred)

            # clf, _ = train_classifier(model, x_train, y_train)
            # y_pred = predict_classifier(model, clf, x_test)
            # score = accuracy_score(y_test, y_pred)
            # append to array of values that are to be returned
            told.append(score)

        snes.tell(asked, told)
        end = timer()
        print("It took", end - start, "seconds to complete generation", i + 1)

    nnw.set_weights(snes.center)

    clf, _ = train_classifier(model, train_imgs, train_labels)
    y_pred = predict_classifier(model, clf, test_imgs)
    print(told)

    print(test_labels.shape, y_pred.shape)
    test_accuracy = accuracy_score(test_labels, y_pred)
    print('Test accuracy:', test_accuracy)
    
    #cm = confusion_matrix(test_labels, y_pred)
    #plt.plot(told)
    #plt.title("loss model")
    #plt.xlabel("genaration")
    #plt.ylabel("loss_value")
    #plt.legend("test",loc = "upper left")
    #plt.show()
if __name__ == '__main__':
    main()

Total number of weights to evolve is: (34815,)
2817


ValueError: Error when checking : expected conv2d_4_input to have 4 dimensions, but got array with shape (2817, 31)